In [1]:
import numpy as np
import random

### 分fold

In [3]:
from sklearn.model_selection import KFold,StratifiedKFold
def split_train_test(X,Y):
    """
    数据集切分
    """
    sfolder = StratifiedKFold(n_splits = 3, random_state = 0, shuffle = False)

    train_indexes = []
    test_indexs = []
    for train, test in sfolder.split(X, Y):
        train_indexes.append(train)
        test_indexs.append(test)

    x_train = [X[i] for i in train_indexes[0]]
    x_test = [X[i] for i in test_indexs[0]]

    y_train =[Y[i] for i in train_indexes[0]]
    y_test = [Y[i] for i in test_indexs[0]]
    return x_train,y_train,x_test,y_test

In [ ]:
def split_backgroud_eval(background_labels,eval_labels,all_pd):
    """"""
    trainpds = []
    testpds = []
    for label in background_labels:
        trainpds.append(all_pd[all_pd["Label"]==label])
    for label in eval_labels:
        testpds.append(all_pd[all_pd["Label"]==label])
    train_pd = pd.concat(trainpds)
    test_pd = pd.concat(testpds)
    return train_pd,test_pd

### 构造数据集

In [1]:
def build_pair_dataset(X,Y,f=None):
    """
    X:data
    Y:label
    build pair of dataset  
    构造数据集
    """
    def convert(z):
        if f:
            z = f(z)
        return z
    lefts = []
    rights = []
    targets = []
    for i in range(len(X)):
        l = convert(X[i])
        for j in range(i,len(X)):
            r = convert(X[j])
            lefts.append(l)
            rights.append(r)
            if Y[i] == Y[j]:
                targets.append(1)
            else:
                targets.append(0) 
    data = {"lefts":np.array(lefts),
           "rights":np.array(rights),
           "targets":np.array(targets)}
    return data

In [33]:
import random
def build_triplet_dataset(X,Y):
    """
    X:data
    Y:label
    for triplet loss 
    构造数据集
    """
    pairs = []
    classes = set(Y)
    Y = np.array(Y)
#     indices = [np.where(y_train == i)[0] for i in range(num_classes)]  # [[]] 每个类别的下标List
    for l in classes:
        p_indices = np.where(Y == l)[0] # [[]] 每个类别的下标List
        n_indices = np.where(Y != l)[0]
        for j in n_indices:
            p_indexs = random.sample(list(p_indices),2)
            pairs += [[X[p_indexs[0]],X[p_indexs[1]],X[j]]]
    return np.array(pairs)

In [37]:
# X = ["1","2","3","11","22","33","111","222","333"]
# Y = [0,0,0,1,1,1,2,2,2]
# pairs = build_triplet_dataset(X,Y)
# print(pairs)
# pairs[:,0]

In [8]:
import numpy as np
def create_pairs(x, digit_indices, num_classes):
    """
    创建正例和负例的Pairs
    :param x: 数据
    :param digit_indices: 不同类别的索引列表
    :param num_classes: 类别
    :return: Triplet Loss 的 Feed 数据
    """

    pairs = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1  # 最小类别数
    print(n)
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z3 = digit_indices[dn][i]
            pairs += [[x[z1], x[z2], x[z3]]]
    return np.array(pairs)


In [38]:
# x = [[1,1],[1,2],[1,3],[1,4],
#      [2,1],[2,2],[2,3],[2,4],
#      [3,1],[3,2],[3,3],[3,4],
#      [4,1],[4,2],[4,3],[4,4]]
# indices = [[0,1,2,3],[4,5,6,7],[8,9,10,11],[12,13,14,15]]
# num_classes = 4
# create_pairs(x, indices, num_classes)

### data load and save

In [ ]:
import pickle
import csv

In [ ]:
# 读取csv文本文件
def filter_empty(n):
    return n != ""

def read_csvdata(data_path):
    cnt = 0
    rows = []
    with open(data_path,mode='r',encoding='utf-8',newline='') as f:
            #此处读取到的数据是将每行数据当做列表返回的
            reader = csv.reader(f)
            for row in reader:
                row = list(filter(filter_empty,row))
                #此时输出的是一行行的列表
                for r in row:
                    rows.append((r,cnt))
                cnt +=1
    return rows

# data_path = "../Dataset/Contract/Synonyms/contract_lx.csv"
# lx = read_csvdata(data_path)

In [ ]:
def pickle_load_data(data_path):
    fr = open(data_path,'rb')  
    data = pickle.load(fr) 
    return data
# data_path = './data/traindata.txt'

In [5]:
def pickle_dump_data(data,data_path):
    #使用dump()将数据序列化到文件中  
    fw = open(data_path,'wb')  
    pickle.dump(data, fw) 

In [ ]:
# data_path = './data/testdata.txt'

### model load and save

In [ ]:
# 保存
from sklearn.externals import joblib
# todump = {
#     "clf":ovr_clf,
#     "tdidfvec":tfidf_vectorizer,
#     "labelenc":label_enc
# }
def sklearn_dump(todump,model_path):
    joblib.dump(todump, "contract_v1.m")